In [1]:
!python -V

Python 3.11.13


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

In [8]:
import mlflow

# Using absolute path to the correct DB
mlflow.set_tracking_uri("http://localhost:5000") 
mlflow.set_experiment("nyc-taxi-experiment")

2025/08/13 09:19:23 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1755076763712, experiment_id='1', last_update_time=1755076763712, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [27]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] +'_'+ df['DOLocationID']
    return df

In [28]:
df_train = pd.read_parquet('/workspaces/mlops-zoomcamp/02-mlflow/data/green_tripdata_2021-01.parquet')
df_val = pd.read_parquet('/workspaces/mlops-zoomcamp/02-mlflow/data/green_tripdata_2021-02.parquet')

In [31]:
df_train['PU_DO'] = df_train['PULocationID'].astype(str) +'_'+ df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) +'_'+ df_val['DOLocationID'].astype(str)

In [32]:
df_train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,PU_DO
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00,43_151
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75,166_239
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00,41_42
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00,168_75
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,...,-0.5,0.00,0.0,None,-0.3,-52.80,3.0,1.0,0.00,265_265


In [33]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [34]:
# Convert string timestamps to datetime if they're not already
df_train['lpep_pickup_datetime'] = pd.to_datetime(df_train['lpep_pickup_datetime'])
df_train['lpep_dropoff_datetime'] = pd.to_datetime(df_train['lpep_dropoff_datetime'])
df_val['lpep_pickup_datetime'] = pd.to_datetime(df_val['lpep_pickup_datetime'])
df_val['lpep_dropoff_datetime'] = pd.to_datetime(df_val['lpep_dropoff_datetime'])

# Calculate duration in minutes
df_train['duration'] = (df_train['lpep_dropoff_datetime'] - df_train['lpep_pickup_datetime']).dt.total_seconds() / 60
df_val['duration'] = (df_val['lpep_dropoff_datetime'] - df_val['lpep_pickup_datetime']).dt.total_seconds() / 60

# Now we can access the duration column
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

PART 2 : COURSE XGBOOST EXAMPLE

In [41]:
import xgboost as xgb
import numpy as np

In [38]:
mlflow.xgboost.autolog(disable=True) # Autolog automatiquement sur MLFLOW - cons : debug diff à isoler le prob, logs perso non dispo avec autolog, controle etc

In [45]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=30,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mlflow.log_metric("rmse", rmse)

    with open("preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, name="models_mlflow")

[0]	validation-rmse:53.00786
[1]	validation-rmse:52.95303
[2]	validation-rmse:52.99269
[3]	validation-rmse:53.11095
[4]	validation-rmse:53.26545
[5]	validation-rmse:53.46482
[6]	validation-rmse:53.63349
[7]	validation-rmse:53.82890
[8]	validation-rmse:54.04723
[9]	validation-rmse:54.24796
[10]	validation-rmse:54.48840
[11]	validation-rmse:54.68821
[12]	validation-rmse:54.90494
[13]	validation-rmse:55.11320
[14]	validation-rmse:55.31020
[15]	validation-rmse:55.51602
[16]	validation-rmse:55.72216
[17]	validation-rmse:55.86983
[18]	validation-rmse:56.05304
[19]	validation-rmse:56.16449
[20]	validation-rmse:56.32612
[21]	validation-rmse:56.44342
[22]	validation-rmse:56.54786
[23]	validation-rmse:56.65801
[24]	validation-rmse:56.79654
[25]	validation-rmse:56.92093
[26]	validation-rmse:56.97124
[27]	validation-rmse:57.03038
[28]	validation-rmse:57.09771
[29]	validation-rmse:57.11133


/home/codespace/miniforge3/envs/new-env/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [09:44:54] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/08/13 09:44:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run learned-quail-473 at: http://localhost:5000/#/experiments/1/runs/7a06d842bfdf447e96fc8991387794a3
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [27]:
import pandas as pd
import mlflow
import xgboost as xgb
import pickle
import os
import numpy as np

params = {    
    'learning_rate': 0.1478262522723516,
    'max_depth': 4,
    'min_child_weight': 13.971201876547578,
    'objective': 'reg:squarederror',
    'reg_alpha': 0.14143510877266188,
    'reg_lambda': 0.23656439070441268,
    'seed': 42
}

with mlflow.start_run():
    mlflow.log_params(params)
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val,y_pred))
    mlflow.log_metric("rmse", rmse)

# Sauvegarde du modèle XGBoost en binaire
    os.makedirs("models", exist_ok=True)
    model_path = "models/xgb_model.bin"
    booster.save_model(model_path)

    # Log avec MLflow
    mlflow.log_artifact(local_path=model_path, artifact_path="models_pickle")

    # Log avec MLflow via log_model
    mlflow.xgboost.log_model(booster, artifact_path="models_xgb")

[0]	validation-rmse:52.85412


[1]	validation-rmse:52.62032
[2]	validation-rmse:52.45100
[3]	validation-rmse:52.32615
[4]	validation-rmse:52.23601
[5]	validation-rmse:52.16531
[6]	validation-rmse:52.11236
[7]	validation-rmse:52.06777
[8]	validation-rmse:52.04211
[9]	validation-rmse:52.02105
[10]	validation-rmse:52.00349
[11]	validation-rmse:51.99251
[12]	validation-rmse:51.99707
[13]	validation-rmse:51.98859
[14]	validation-rmse:51.98114
[15]	validation-rmse:51.98108
[16]	validation-rmse:51.97805
[17]	validation-rmse:51.98183
[18]	validation-rmse:51.97667
[19]	validation-rmse:51.98445
[20]	validation-rmse:51.98418
[21]	validation-rmse:51.98663
[22]	validation-rmse:51.99053
[23]	validation-rmse:51.99053
[24]	validation-rmse:51.99693
[25]	validation-rmse:51.99764
[26]	validation-rmse:51.99896
[27]	validation-rmse:52.00547
[28]	validation-rmse:52.00756
[29]	validation-rmse:52.00600
[30]	validation-rmse:52.00987
[31]	validation-rmse:52.02012
[32]	validation-rmse:52.02788
[33]	validation-rmse:52.02972
[34]	validation-rms

/tmp/ipykernel_2631/3290792332.py:34: UserWarning: [12:34:38] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  booster.save_model(model_path)
2025/08/01 12:34:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/miniforge3/envs/new-env/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [12:34:38] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/08/01 12:34:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
